In [1]:
from nltk.stem.porter import PorterStemmer
import numpy as np

In [2]:
#数据读取
def ReadData(text):
    data = []
    for line in text.readlines():
        data.append([eval(line).get('tweetId'),eval(line).get('text')]) #读取使用的tweetId和text两项
    return data

In [3]:
#数据预处理
def DataPreprocess(data_lists): #参数是一个二维列表，每一项表示为[id,text]
    file = []    
    unused_words_file = open('unused.txt','r')
    unused_words = unused_words_file.read()
    unused_words_file.close()
    unused_words_list = unused_words.split()
    for data_list in data_lists: 
        #全部变成小写
        data_list[1] = data_list[1].lower()
        # 特殊符号替换：可以保留一部分网址、邮箱等特殊词项
        # 删除所有的逗号、括号、其他特殊符号
        for ch in '"#$%&()*+,-—–;<=>[]^_‘{|}~“”""':
            data_list[1] = data_list[1].replace(ch,' ')         
        for s in ['…','..','...','....','.....','......','. ',' .',': ',"' "," '","''"]:            
            data_list[1] = data_list[1].replace(s,' ')
            
        # 特殊位置替换
        words_list = data_list[1].split()
        for j,item in enumerate(words_list):
            if item[0] in "?'./!":
                words_list[j] = item.replace(item[0],'')
            if item[-1] in "?'./!":
                words_list[j] = item.replace(item[-1],'')
        data_list[1] = ' '.join(words_list)
        t = data_list[1].split(' ')
        t.insert(0,data_list[0])
        file.append(t)
        
    # 去除停用词
    for i,words in enumerate(file):
        for j in unused_words_list:
            if j in words:
                words.pop(words.index(j))                    
    # 词项归一化，暂时不作处理
    
    # 词干还原 
    porter_stemmer = PorterStemmer()  
    for i,line in enumerate(file):
        for j,word in enumerate(line):
            line[j] = porter_stemmer.stem(word)
        file[i] = line  
          
    file.sort() #好像没什么用 原本的tweets就是有序的
    return file

In [4]:
#建立倒排索引
def get_inverted_index(file):
    inverted_index = {}
    tweet_dicts_list = []
    #建立每个tweet的词项字典
    for tweet in file:
        tweet[0] = eval(tweet[0]) #因为前面要过词干提取，所以那时候id要用字符串
        tweet_dict = {}
        for i,item in enumerate(tweet):
            if i==0:
                continue
            else:  #删掉空的词项
                if item == '':
                    continue
                #对于每一个词项，在该文档出现的次数记下来
                if item not in tweet_dict.keys():
                    tweet_dict[item] = [1,tweet[0]] #因为要动态修改 所以不能用元组
                else:
                    tweet_dict[item][0] += 1
        tweet_dicts_list.append(tweet_dict)
    #对词项字典进行合并 这个版本的倒排索引中就不记录频率了
    for tweet_dict in tweet_dicts_list:
        for key in tweet_dict:
            if key not in inverted_index.keys():
                inverted_index[key] = []
                inverted_index[key].append(tweet_dict[key])
            else:
                inverted_index[key].append(tweet_dict[key])
    #对输出的倒排索引排序 根据key的字母序 字典肯定没办法排序的
    sorted_inverted_index_keys = sorted(inverted_index.keys())
    sorted_inverted_index_list = []
    for key in sorted_inverted_index_keys:        
        sorted_inverted_index_list.append([[key,len(inverted_index[key])],inverted_index[key]])  
    return sorted_inverted_index_list

In [5]:
tweets_open = open('tweets.txt','r')
data_ori = ReadData(tweets_open)
tweets_open.close()
N = len(data_ori)
id_list_file = open('all_text.txt','w',encoding='utf-8')
id_list_file.write(str(data_ori))
id_list_file.close()
data = DataPreprocess(data_ori)
print(data[:20])
inverted_index= get_inverted_index(data)
inverted_index_file = open('inverted_index.txt','w',encoding='utf-8')
inverted_index_file.write(str(inverted_index))
inverted_index_file.close()
print(inverted_index[12010:12015])

[['28965792812892160', 'hous', 'may', 'kill', 'arizona', 'style', 'immigr', 'bill', 'rep', 'rick', 'rand', 'say', 'hous', 'unlik', 'pass', 'the', 'ari', 'http://tinyurlcom/4jrjcdz'], ['28967095878287360', 'mourner', 'recal', 'sarg', "shriver'", 'chariti', 'ideal', 'ap', 'ap', 'r', 'sargent', 'shriver', 'alway', 'optimist', 'pio', 'http://bit.ly/gqmcdg'], ['28967672074993664', 'bass', 'fish', 'techniqu', '2', 'fantast', 'tip', 'improv', 'your', 'cast', 'skill'], ['28967914417688576', 'financi', 'aid', 'proper', 'method', 'get', 'financi', 'aid', 'educ', 'http://ping.fm/bk0r3', 'appli', 'for', 'financi', 'aid', 'financi', 'aid', 'essay'], ['28968479176531969', 'suprem', 'court', "nasa'", 'intrus', 'background', 'check', 'ok', 'http://bit.ly/h2jgy9'], ['28968581949558787', 'mcdonald', 'music', 'firework', 'all', 'time', 'low'], ['28969422056071169', '@alyc', 'veri', 'sweet', 'quiet', 'if', 'not', 'polish', 'bono', 'hansard', 'sgt', "shriver'", 'funer', '2day', 'http://youtu.be/bf14xbbcvzg

inverted_index = eval(open('inverted_index.txt','r',encoding='utf-8').read())
print(inverted_index[12010:12015])

In [6]:
df = []
for i in inverted_index:
    df.append(i[0][1])
df = np.array(df)

In [7]:
def id_vector(data):
    id_dic = {}
    for i in data:
        id_dic[i[0]] = 0
    return id_dic

In [8]:
def inverted_index_vector(inverted_index):
    inverted_index_dic = {}
    for i in inverted_index:
        inverted_index_dic[i[0][0]] = 0
    return inverted_index_dic

In [9]:
def tf_log(array):
    tf_log_list = []
    for i in array:
        if i == 0:
            tf_log_list.append(0)
        else:
            tf_log_list.append(1+np.log10(i))
    return np.array(tf_log_list)

In [10]:
def option_check(option):
    std_option = ['nlabL','ntp','nc']
    if len(option) == 3:
        for i in range(3):
            if option[i] in std_option[i]:
                pass
            else:
                return False
        return True
    else:
        return False

In [30]:
#ALL SMART notations
def doc_vector(df, term_list, inverted_index, option):#将文档根据倒排索引转化成一个稀疏的向量 有归一化
    term_dict = inverted_index_vector(inverted_index)
    tf_raw = []
    id_list = []
    for term in term_list:#对于每个查询词项
        #计算term在term_list中的出现次数:tf-raw
        if term not in term_dict.keys():
            pass #对于所有文档中都没有出现过的词忽略不计
        else:
            term_dict[term] += 1
    for i in inverted_index:
        if i[0][0] in term_list:
            for j in i[1]:
                id_list.append(j[1])
    for key in term_dict:
        tf_raw.append(term_dict[key])
    option_list = [[np.array(tf_raw),tf_log(np.array(tf_raw)),(0.5 + (0.5*np.array(tf_raw)/max(tf_raw))),np.sign(tf_raw),(tf_log(np.array(tf_raw))/tf_log(np.full_like(np.array(tf_raw),np.mean(tf_raw),dtype = 'float')))],
                  [1,tf_log(N/np.array(df)),np.maximum(0,tf_log((N - np.array(df))/np.array(df)))],
                  [1,1 / np.linalg.norm(np.array(tf_raw))]]

    #option是一个字符串
    std_option = ['nlabL','ntp','nc']
    result = np.ones_like(tf_raw)
    for i in range(3):
        result =result * np.array(option_list[i][std_option[i].find(option[i])])
    return result,id_list

In [31]:
#时间复杂度略高
def cosine_score(df, op_q,op_d, query, document_list, inverted_index): #用预处理后的list
    scores = []#存放每篇文档的得分
    query_vector,id_list = doc_vector(df, query, inverted_index, op_q)
    for i in document_list:
        if i[0] in id_list:
            document_vector,unuse = doc_vector(df, i[1:], inverted_index, op_d)
            scores.append([i[0],sum(query_vector*document_vector)/len(i)])    
    return sorted(scores,key = lambda x:x[1],reverse=True)

In [32]:
query = input('请输入查询语句：')
query_list = DataPreprocess([['0',query]])[0][1:]
option_doc = input('请输入文档处理模式：')
while not option_check(option_doc):
    option_doc = input('输入错误，请重新输入：')
option_query = input('请输入查询处理模式：')
while not option_check(option_query):
    option_query = input('输入错误，请重新输入：')
qualified_quary_list = []
id_list = []
score_list = cosine_score(df, option_query, option_doc, query_list, data[:2000], inverted_index)
for i in score_list:
    id_list.append(str(i[0]))
for i in data_ori:
    if i[0] in id_list:
        qualified_quary_list.append(i[1])
print('共找到符合条件的结果{}条，评分最高的前10条如下'.format(len(qualified_quary_list)))
for i in qualified_quary_list[:10]: #K
    print(i)

请输入查询语句：a big house
请输入文档处理模式：anc
请输入查询处理模式：bnn
共找到符合条件的结果50条，评分最高的前10条如下
house may kill arizona style immigration bill rep rick rand says the house is unlikely to pass the ari http://tinyurlcom/4jrjcdz
oprah's family secret weekly world news the weekly world news will not wait for the big o to divulge her litt http://bit.ly/id1qio
new easy recipe ranch house casserole http://bit.ly/htangc recipes
producers guild gives the king's speech the big win http://st.ep2.tv/fbhsx2 film
eric cantor addresses birther issue washington the new republican house majority leader says he doesn't thi http://twurl.nl/lr74ji
dtn world news cantor i believe obama is a u.s citizen the new republican house majority leader says he does http://bit.ly/g6n09o
understatement 8/5 runs in race 8 at big a for a 50g tag 6 year old horse with class but the tag sends mixed signals willing to lose him
chinese pianist plays anti american tune at white house the blaze http://goo.gl/zu4jj
@apoloohno if that horse races bet